In [59]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import numpy as np
import pandas as pd
import sys,os,math
import time
from datetime import datetime
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Load train Data
dir_name = "/Users/jordan/Project/DataMining/traffic_dataset/train_set/"
df = pd.DataFrame()
for file_path in os.listdir(dir_name):
    os.chdir(dir_name + file_path)
    for file_name in os.listdir():
        new_df = pd.read_csv(file_name, header = None, dtype = np.unicode_)
        df = df.append(new_df)
        
#         print(file_name)
    
# file_name = '0707_seg_1.txt'

In [22]:
# Clean traffic Data
# Initialize the Data
# Data = pd.read_csv(filename, header = None, dtype = np.unicode_)
Data = df.drop([11], axis = 1)
Data.columns = ['date','time','direction','type','linkID','length','travelTime','volumn','speed','occupancy','congestionLevel']
Data = Data.drop(columns = ['date'])
Data = Data.dropna()
# Data = Data[:1000]

#  Vectorize the parameters
Data.length = np.float64(Data.length)
Data.travelTime = np.float64(Data.travelTime)
Data.volumn = np.float64(Data.volumn)
Data.speed = np.float64(Data.speed)
Data.occupancy = np.float64(Data.occupancy)

Data = Data[Data['length'] >= 0]
Data = Data[Data['travelTime'] >= 0]
Data = Data[Data['volumn'] >= 0]
Data = Data[Data['speed'] >= 0]

Data = Data.reset_index()
occpcy_mode = np.float64(Data[Data['occupancy'] >= 0 ].occupancy.mode())
for j in range(len(Data)):
    if Data.loc[j,'occupancy'] < 0.:
        Data.loc[j, 'occupancy'] = occpcy_mode
Data.occupancy =  preprocessing.scale(Data.occupancy.values.reshape(-1,1))
Data = Data.drop(columns = ['index'])

# for i in range(len(Data)):
# #     Time = Data.loc[i,'time']
# #     time_strp = datetime.strptime(Time, '%H%M')
# #     Data.loc[i,'time'] = time_strp.hour + time_strp.minute/60
    
#     sys.stdout.write('   \r')
#     sys.stdout.flush()
#     sys.stdout.write('{}%\r'.format(i))
#     sys.stdout.flush()
#     time.sleep(0.01)
    
# Data.time = preprocessing.normalize(Data.time.values.reshape(-1,1), norm = 'max')

ID = list(Data.linkID.unique())
Time = list(Data.time.unique())
Data['direction'] = Data['direction'].map({'EAST_BOUND':0, 'WEST_BOUND':1, 
                                               'SOUTH_BOUND':2, 'NORTH_BOUND':3, 'STH_BOUND':4, 'UNKNOWN_DIRECTION_TYPE':5}).astype(int)
Data['type'] = Data['type'].map({'FREEWAY': 0,'RAMP': 1,'ARTERIAL': 2,'LOCAL_ROAD':3, 
                                     'FREEWAY_REVERSIBLE': 4,'FREEWAY_EXPRESS': 5}).astype(int)
Data['congestionLevel'] = Data['congestionLevel'].map({'NON_CONGESTION': 0,'LIGHT_CONGESTION': 1,'MEDIUM_CONGESTION': 2,
                                                         'HEAVY_CONGESTION': 3,'UNKNOWN_CONGESTION_LEVEL': 4 }).astype(int)                                     
Data['linkID'] = Data['linkID'].map(dict(zip(ID, np.arange(len(ID))))).astype(int)
Data['time'] = Data['time'].map(dict(zip(Time, np.arange(len(Time))))).astype(int)

Data = Data.drop(Data[Data['congestionLevel'] == 4].index)


length_norm = np.array(Data.length).reshape(-1, 1)
trvTime_norm = np.array(Data.travelTime).reshape(-1, 1)
volumn_norm = np.array(Data.volumn).reshape(-1, 1)
speed_norm = np.array(Data.speed).reshape(-1, 1)
occu_norm = np.array(Data.occupancy).reshape(-1, 1)

est_length = KBinsDiscretizer(n_bins=100, encode='ordinal', strategy='uniform').fit(length_norm)
est_trvTime = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='uniform').fit(trvTime_norm)
est_volumn = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='uniform').fit(volumn_norm)
est_speed = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform').fit(speed_norm)
est_occu = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform').fit(occu_norm)

length_norm = est_length.transform(length_norm)
trvTime_norm = est_trvTime.transform(trvTime_norm)
volumn_norm  = est_volumn.transform(volumn_norm)
speed_norm = est_speed.transform(speed_norm)
occu_norm = est_occu.transform(occu_norm)

Data.length = length_norm
Data.travelTime = trvTime_norm
Data.volumn = volumn_norm
Data.speed = speed_norm
Data.occupancy = occu_norm

In [24]:
# Y_norm = np.array(Data['congestionLevel']).reshape(-1, 1)
# binarizer = preprocessing.Binarizer().fit(Y_norm)
# Y_train = binarizer.transform(Y_norm)
X_train = Data.iloc[:, 0:9]
Y_train = Data['congestionLevel']


In [28]:
# Model
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

/Users/jordan/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
print(clf.feature_importances_)

[0.08068901 0.01015818 0.00511394 0.06104931 0.03835712 0.03789549
 0.03658647 0.71623176 0.01391871]


In [31]:
X_train.tail()

,time,direction,type,linkID,length,travelTime,volumn,speed,occupancy
717034,71,0,0,810,4.0,0.0,6.0,6.0,0.0
717035,71,3,0,811,22.0,4.0,0.0,5.0,0.0
717036,71,2,0,812,6.0,1.0,0.0,5.0,0.0
717037,71,1,0,813,0.0,0.0,2.0,6.0,0.0
717038,71,2,0,814,93.0,15.0,0.0,6.0,0.0


In [36]:
print(clf.predict([[0, 2, 0, 1, 0.0, 0.0, 28.0, 3.0, 2.0]]))

[2]


In [35]:
X_train.head()

,time,direction,type,linkID,length,travelTime,volumn,speed,occupancy
0,0,3,0,0,2.0,0.0,17.0,6.0,0.0
1,0,2,0,1,0.0,0.0,28.0,3.0,2.0
2,0,1,0,2,6.0,1.0,0.0,5.0,0.0
3,0,0,0,3,16.0,3.0,0.0,5.0,0.0
4,0,1,0,4,1.0,0.0,33.0,5.0,1.0


In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0721,1800,NORTH_BOUND,FREEWAY,WI-MNT_XML_V001-21012,1268,40,1025,31.292915,7.25,NON_CONGESTION,NaN
1,0721,1800,SOUTH_BOUND,FREEWAY,WI-MNT_XML_V001-25006,476,30,1660,15.44976,21.37,MEDIUM_CONGESTION,NaN
2,0721,1800,WEST_BOUND,FREEWAY,WI-MNT_XML_V001-240,4124,150,0,27.0,-1.0,NON_CONGESTION,NaN
3,0721,1800,EAST_BOUND,FREEWAY,IL-TESTTSC-202,9718,390,0,24.0,-1.0,LIGHT_CONGESTION,NaN
4,0721,1800,WEST_BOUND,FREEWAY,WI-MNT_XML_V001-31013,830,30,1979,26.8225,11.88,NON_CONGESTION,NaN
